In [1]:
# make an abstract compiler
from pyquil import Program
from pyquil.api._qac import AbstractCompiler
class DummyCompiler(AbstractCompiler):
    def get_version_info(self):
        return {}
    def quil_to_native_quil(self, program: Program):
        return program
    def native_quil_to_executable(self, nq_program: Program):
        return nq_program

In [2]:
#make a quantum computer object
import networkx as nx
from pyquil.device import NxDevice
from pyquil.api import QVM, QuantumComputer
from pyquil.pyqvm import PyQVM
from pyquil.reference_simulator import ReferenceDensitySimulator

device = NxDevice(nx.complete_graph(1))
qc = QuantumComputer(name='testy!',
                     qam=PyQVM(n_qubits=1,quantum_simulator_type=ReferenceDensitySimulator),
                     device=device,
                     compiler=DummyCompiler())

**make some programs**

In [3]:
from pyquil.gates import I, MEASURE

# run prog
prog = Program(I(0))
ro = prog.declare('ro', 'BIT', 1)
prog += MEASURE(0, ro[0])

# wrap in numshots prog
progwrap = prog.copy()
progwrap.wrap_in_numshots_loop(10)
print(prog)

# Run and measure style
progRAM = Program(I(0))
print(progRAM)

I 0
DECLARE ro BIT[1]
MEASURE 0 ro[0]

I 0



# Does not work if you just set density

In [4]:
# inspect the state of the QVM
print('the old state\n', qc.qam.wf_simulator.density)

# density matrix 
import numpy as np
rho1 = np.array([[ 0.0, 0.0], [0.0,  1.0]])
print('the new state\n',rho1)

the old state
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
the new state
 [[0. 0.]
 [0. 1.]]


In [5]:
# load mixed state into QVM
qc.qam.wf_simulator.density = rho1
# display QVM state
print(qc.qam.wf_simulator.density)

[[0. 0.]
 [0. 1.]]


In [6]:
# run
qc.qam.wf_simulator.density = rho1
print(qc.run(prog))
#  
for _ in range(0,4):
    print(qc.run(prog))

[[1]]
[[0]]
[[0]]
[[0]]
[[0]]


In [7]:
# Run and measure
qc.qam.wf_simulator.density = rho1

print(qc.qam.wf_simulator.density)
print(qc.run_and_measure(progRAM,trials=40))
print(qc.qam.wf_simulator.density)
print(qc.run_and_measure(progRAM,trials=40))

[[0. 0.]
 [0. 1.]]
{0: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
{0: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}


# Works if you `qc.qam.wf_simulator.set_density`

In [8]:
qc.qam.wf_simulator.set_density(rho1)

In [9]:
# run
print(qc.run(prog))
#  
for _ in range(0,4):
    print(qc.run(prog))


[[1]]
[[1]]
[[1]]
[[1]]
[[1]]


In [10]:
# run and measure

print(qc.qam.wf_simulator.density)
print(qc.run_and_measure(progRAM,trials=10))
print(qc.qam.wf_simulator.density)
print(qc.run_and_measure(progRAM,trials=40))

[[0. 0.]
 [0. 1.]]
{0: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
[[0. 0.]
 [0. 1.]]
{0: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}


# density works with `measure_observables` if `calibrate_readout=None`

In [11]:
from forest.benchmarking.tomography import generate_state_tomography_experiment
from forest.benchmarking.tomography import iterative_mle_state_estimate, linear_inv_state_estimate

In [12]:
# check if the density is correct
print(qc.qam.wf_simulator.density)

[[0. 0.]
 [0. 1.]]


In [13]:
experiment_1q = generate_state_tomography_experiment(program=progRAM, qubits=[0])

from pyquil.operator_estimation import ExperimentSetting, TensorProductState, TomographyExperiment
from pyquil.paulis import sZ

print(experiment_1q)

I 0
0: Z0_0→(1+0j)*I
1: Z0_0→(1+0j)*X0
2: Z0_0→(1+0j)*Y0
3: Z0_0→(1+0j)*Z0


In [14]:
from pyquil.operator_estimation import measure_observables


resultsy = list(measure_observables(qc=qc, tomo_experiment=experiment_1q, n_shots=8000, calibrate_readout=None))
resultsy

[ExperimentResult[Z0_0→(1+0j)*I: 1.0 +- 0.0],
 ExperimentResult[Z0_0→(1+0j)*X0: 0.0025 +- 0.011180304948882207],
 ExperimentResult[Z0_0→(1+0j)*Y0: -0.01325 +- 0.011179358420209095],
 ExperimentResult[Z0_0→(1+0j)*Z0: -1.0 +- 0.0]]

**estimate the state**

In [15]:

rho = iterative_mle_state_estimate(results=resultsy, qubits=[0])
print('rho est', np.around(rho, decimals=2))
print('===')
print('rho true', np.around(rho1, decimals=2))


rho est [[0.+0.j 0.+0.j]
 [0.-0.j 1.-0.j]]
===
rho true [[0. 0.]
 [0. 1.]]


In [16]:
linear_inv_state_estimate(results=resultsy, qubits=[0])

array([[-1.11022302e-16+0.j      ,  1.25000000e-03+0.006625j],
       [ 1.25000000e-03-0.006625j,  1.00000000e+00+0.j      ]])